In [1]:
from astropy.table import Table
import sys, json
import numpy as np

try: # Python 3.x                                                                                                            
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
    import http.client as httplib
except ImportError: # Python 2.x                                                                                             
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve
    import httplib

In [2]:
def crossmatch(pos, service, r):
    crossmatchInput = {'fields': [{'name':'ra' , 'type':'float'},
                                      {'name':'dec', 'type':'float'}],
                           'data': [{'ra':pos[0], 'dec':pos[1]}]}
    request = {'service':service,
                   'data':crossmatchInput,
                   'params': {'raColumn':'ra', 'decColumn':'dec', 'radius':r},
                   'format':'json'}

    headers, outString = mastQuery(request)
    print(jsonTable(json.loads(outString)))
    return jsonTable(json.loads(outString))

In [3]:
 def mastQuery(request):
        """                                                                                                                  
        Sends a request to the MAST server                                                                                   
        Parameters                                                                                                           
        ----------                                                                                                           
            request: json string                                                                                             
        Returns                                                                                                              
        ----------                                                                                                           
            head: headers for response                                                                                       
            content: data for response                                                                                       
        """
        server = 'mast.stsci.edu'

        # Grab Python Version                                                                                                
        version = '.'.join(map(str, sys.version_info[:3]))
        # Create Http Header Variables                                                                                       
        headers = {'Content-type': 'application/x-www-form-urlencoded',
                   'Accept': 'text/plain',
                   'User-agent': 'python-requests/'+version}
        # Encoding the request as a json string                                                                              
        requestString = urlencode(json.dumps(request))
        # Opening the https cnnection                                                                                        
        conn = httplib.HTTPSConnection(server)
        # Making the query                                                                                                   
        conn.request('POST', '/api/v0/invoke', 'request='+requestString, headers)

        # Getting the response                                                                                               
        resp = conn.getresponse()
        head = resp.getheaders()
        content = resp.read().decode('utf-8')

        # Close the https connection                                                                                         
        conn.close()

        return head, content

In [4]:
def jsonTable(jsonObj):
    dataTable = Table()
    for col,atype in [(x['name'],x['type']) for x in jsonObj['fields']]:
            if atype=='string':
                atype='str'
            if atype=='boolean':
                atype='bool'
            if atype=='int':
                atype='float'
            dataTable[col] = np.array([x.get(col,None) for x in jsonObj['data']],dtype=atype)
    return dataTable


In [7]:
pos = [[271.08531128857544, 42.91121805498896],
        [271.08534567510867, 42.91162646277188],
       [271.093164974481, 42.901744900855846],
       [271.0900966477034, 42.895771644048935],
       [271.09976967986245, 42.89034147460848],
       [271.0774059919819, 42.941862135778194]]

r = 0.1
service = 'Mast.Tic.Crossmatch'

for i in pos:
    table = crossmatch(i, service, r)

       ra              dec         MatchID  ...    priority   _selected_
---------------- ---------------- --------- ... ------------- ----------
271.085345675109 42.9116264627719 198593129 ... 0.00364195555      False
271.085345675109 42.9116264627719 198593127 ...           nan      False
271.085345675109 42.9116264627719 198593130 ...           nan      False
271.085345675109 42.9116264627719 198593125 ...           nan      False
271.085345675109 42.9116264627719 198593124 ...           nan      False
271.085345675109 42.9116264627719 198593120 ...           nan      False
271.085345675109 42.9116264627719 198593117 ...           nan      False
271.085345675109 42.9116264627719 198593115 ...           nan      False
271.085345675109 42.9116264627719 198593122 ...           nan      False
271.085345675109 42.9116264627719 198593119 ...           nan      False
             ...              ...       ... ...           ...        ...
271.085345675109 42.9116264627719 198592058 ...    

       ra              dec         MatchID  ...    priority   _selected_
---------------- ---------------- --------- ... ------------- ----------
271.085345675109 42.9116264627719 198593129 ... 0.00364195555      False
271.085345675109 42.9116264627719 198593127 ...           nan      False
271.085345675109 42.9116264627719 198593130 ...           nan      False
271.085345675109 42.9116264627719 198593125 ...           nan      False
271.085345675109 42.9116264627719 198593124 ...           nan      False
271.085345675109 42.9116264627719 198593120 ...           nan      False
271.085345675109 42.9116264627719 198593117 ...           nan      False
271.085345675109 42.9116264627719 198593115 ...           nan      False
271.085345675109 42.9116264627719 198593122 ...           nan      False
271.085345675109 42.9116264627719 198593119 ...           nan      False
             ...              ...       ... ...           ...        ...
271.085345675109 42.9116264627719 198592058 ...    